# Practial 2: Optical Remote Sensing

In this practical, we'll visualize and analyze Landsat 8 imagery data from difference sites.

---

## Import the required libraries

We'll start by importing the things we'll need to load the images, do some calculations, and plot them.

In [ ]:
import skimage.io
import skimage.exposure
import numpy as np
import matplotlib.pyplot as plt

---

## Plotting composites

Load the bands that we'll need from city of Cláudia, Brazil, in 2014.

As a useful reference, here are the Landsat 8 bands:

| Band | Wavelength (µm) | Resolution (m) |
|------|:---------------:|:--------------:|
| 1-Ultra Blue  | 0.435-0.451  | 30 |
| 2-Blue  | 0.452-0.512  | 30 |
| 3-Green  | 0.533-0.590  | 30 |
| 4-Red  | 0.636-0.673  | 30 |
| 5-NIR  | 0.851-0.879  | 30 |
| 6-SWIR 1  | 1.566-1.651  | 30 |
| 7-SWIR 2  | 2.107-2.294  | 30 |
| 8-Panchromatic  | 0.503-0.676  | 15 |
| 9-Cirrus  | 1.363-1.384  | 30 |
| 10-Thermal 1  | 10.60-11.19  | 100 |
| 11-Thermal 2  | 11.50-12.51  | 100 |

We want to make an RGB composite. Which bands do we need?

### Your turn

Make a color-infrared composite of the same scene. What does it highlight and why?

---

## NDVI

Calculate the NDVI for the 2014 image.

---

## Tracking changes with time

Load the images from 2020 so we can visualize the changes.

### Your turn

Calculate the NDVI for 2020 and plot it along side the 2014 NDVI. Can you spot the differences?

---

## Caveats for tracking changes

There are some things to keep in mind for tracking changes with time. To exemplify, let's calculate the difference in NDVI between 2014 and 2020.

---

## Normalized Burn Ratio

Load the images and calculate the NBR for the 2017 Thomas fire in the city of Ventura, USA. We have images from before and after the fire.

### Your turn

Calculate and plot the difference in NBR: dNBR = NBR_before - NBR_after. 

Estimate the area that was severely burned (large dNBR values).

**What are the sources of uncertainty in this calculation?**